In [9]:
#iport libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.models import Model

In [10]:
#Load dataset
df = pd.read_csv('dataset.csv', header=None)

In [11]:
#Define hyperparameters
timesteps = 50 # number of timesteps to include in each input sequence
latent_dim = 16 # number of neurons in the LSTM layer
mae_threshold = 0.05 # threshold for mean absolute error (MAE)
anomaly_window = 20 # length of anomaly window in seconds

In [12]:
#Normalize data
data = np.array(df[0])
mean = np.mean(data)
std = np.std(data)
data = (data - mean) / std

In [13]:
#Create input sequences
sequences = []
for i in range(len(data) - timesteps):
    sequences.append(data[i:i+timesteps])
sequences = np.array(sequences)

In [14]:
#Define autoencoder model
inputs = Input(shape=(timesteps, 1))
encoded = LSTM(latent_dim)(inputs)
decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(1, return_sequences=True)(decoded)
autoencoder = Model(inputs, decoded)

In [15]:
#Compile model
autoencoder.compile(optimizer='adam', loss='mae')

In [16]:
#Train model
history = autoencoder.fit(sequences, sequences, epochs=16, batch_size=32, shuffle=False)

Epoch 1/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1895
Epoch 2/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1718
Epoch 3/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1465
Epoch 4/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1558
Epoch 5/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1450
Epoch 6/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1406
Epoch 7/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1416
Epoch 8/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1377
Epoch 9/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1405
Epoch 10/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1379
Epoch 11/16
483/483 [==============================] - 8s 16ms/step - loss: 0.1383
Epoch 12/16
483/483 [==============================] - 7s 15ms/step - loss: 0.1352
Epoch 13/16
4

In [17]:
#Make predictions
predictions = autoencoder.predict(sequences)

In [18]:
#Calculate MAE
mae = np.mean(np.abs(predictions - sequences), axis=1)

ValueError: operands could not be broadcast together with shapes (15450,50,1) (15450,50) 